# CP 8 Aprendizaje de Máquinas
---

## Redes Neuronales

In [ ]:
import sklearn
import tensorflow as tf
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


### Ejercicio 1: Funciones de Activación

Las funciones de activación se utilizan para determinar la salida de una red neuronal. Mapea los valores resultantes entre 0 a 1 o -1 a 1, etc. (dependiendo de la función). Las funciones de activación son muy importantes porque representan las relaciones no lineales entre las capas de una red neuronal.

Un ejemplo de función de activación es la función *sigmoid*. La curva de la función *sigmoid* parece una forma de S. Va entre 0 y 1, su fórmula es: $$S(z) = \dfrac{1}{1 + e^{-z}}$$

In [ ]:
def sigmoid(z):
    # TODO: Your code here!
    pass

ReLU es la función de activación más utilizada en el mundo en este momento. Ya que, se utiliza en casi todas las redes neuronales convolucionales o de aprendizaje profundo por las facilidades que ofrece el cálculo de su derivada. Su fórmula es: $$R(z)= \max(0, z)$$ 

In [ ]:
def relu(z):
    # TODO: Your code here!
    pass

La siguiente función, calcula la derivada de una función `f` en un punto `z` con una precisión `eps` especificado.

In [ ]:
def derivative(f, z, eps=0.000001):
    return (f(z + eps) - f(z - eps))/(2 * eps)

Vamos a graficar las funciones de activación más populares y sus derivadas.

In [ ]:
z = np.linspace(-5, 5, 200)

plt.figure(figsize=(11,4))

plt.subplot(121)
plt.plot(z, np.sign(z), "r-", linewidth=1, label="Step")
plt.plot(z, sigmoid(z), "g--", linewidth=2, label="Sigmoid")
plt.plot(z, np.tanh(z), "b-", linewidth=2, label="Tanh")
plt.plot(z, relu(z), "m-.", linewidth=2, label="ReLU")
plt.grid(True)
plt.legend(loc="center right", fontsize=14)
plt.title("Activation functions", fontsize=14)
plt.axis([-5, 5, -1.2, 1.2])

plt.subplot(122)
plt.plot(z, derivative(np.sign, z), "r-", linewidth=1, label="Step")
plt.plot(0, 0, "ro", markersize=5)
plt.plot(0, 0, "rx", markersize=10)
plt.plot(z, derivative(sigmoid, z), "g--", linewidth=2, label="Sigmoid")
plt.plot(z, derivative(np.tanh, z), "b-", linewidth=2, label="Tanh")
plt.plot(z, derivative(relu, z), "m-.", linewidth=2, label="ReLU")
plt.grid(True)
#plt.legend(loc="center right", fontsize=14)
plt.title("Derivatives", fontsize=14)
plt.axis([-5, 5, -0.2, 1.2])

plt.show()

### Ejercicio 2: Análisis del Dataset

Primero vamos a importar TensorFlow y Keras.

In [ ]:
import tensorflow as tf
from tensorflow import keras

Comencemos cargando el conjunto de datos MNIST de moda (`fashion_mnist`). Keras tiene una serie de funciones para cargar conjuntos de datos populares en `keras.datasets`. Sin embargo, vamos a cargar estos datos de forma local.

In [ ]:
# This is how you would download the dataset with keras (if you want use it)
# fashion_mnist = keras.datasets.fashion_mnist
# (X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

Para convertir los datos comprimidos en arrays de numpy creamos dos funciones: `load_dataset_data` y `load_dataset_labels`.

In [ ]:
import gzip

In [ ]:
def load_dataset_data(filename: str, num_images: int) -> np.ndarray:
    image_size = 28
    with gzip.open(filename, "r") as f:
        f.read(16)
        buf = f.read(image_size*image_size*num_images)
    data = np.frombuffer(buf, dtype=np.uint8)
    return data.reshape(num_images, image_size, image_size)


In [ ]:
def load_dataset_labels(filename: str, num_images) -> np.ndarray:
    with gzip.open(filename, 'r') as f:
        f.read(8)
        buf = f.read(num_images)
        labels = np.frombuffer(buf, dtype=np.uint8)
    return labels

Carguemos ahora los datos locales:

In [ ]:
X_train_full = load_dataset_data("./resources/fashion-mnist/train-images-idx3-ubyte.gz", 60000)
X_test = load_dataset_data("./resources/fashion-mnist/t10k-images-idx3-ubyte.gz", 10000)
y_train_full = load_dataset_labels("./resources/fashion-mnist/train-labels-idx1-ubyte.gz", 60000)
y_test = load_dataset_labels("./resources/fashion-mnist/t10k-labels-idx1-ubyte.gz", 10000)

El conjunto de entrenamiento contiene 60.000 imágenes en escala de grises, cada una de 28x28 píxeles. Esto lo podemos comprobar viendo las dimensiones de la matriz de características.

In [ ]:
# TODO: Your code here!

Cada intensidad de píxel se representa como un byte (0 a 255). Esto lo podemos comprobar viendo el tipo de datos por el que están compuestos la matriz de características.

In [ ]:
# TODO: Your code here!

Dividamos el conjunto de entrenamiento completo en un conjunto de validación y un conjunto de entrenamiento (más pequeño). 

In [ ]:
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

Luego escalamos las intensidades de los píxeles hasta el rango 0-1 y las convertimos en flotantes, dividiéndolas por 255.

In [ ]:
X_valid = X_valid / 255.
X_train = X_train / 255.
X_test = X_test / 255.

Puede graficar una imagen usando la función `imshow()` de `Matplotlib`, con un mapa de color `'binary'` (binario):

In [ ]:
plt.imshow(X_train[0], cmap="binary")
plt.axis('off')
plt.show()

Las etiquetas son los ID de clase (representados como `uint8`), del 0 al 9:

In [ ]:
# Show the labels
# TODO: Your code here!

Aquí están los nombres de clase correspondientes:

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

¿Cómo podríamos ver cuál es el nombre de la clase de la primera instancia de entrenamiento?

In [ ]:
# TODO: Your code here!

El conjunto de validación contiene 5000 imágenes y el conjunto de prueba contiene 10 000 imágenes:

In [ ]:
# TODO: Your code here!

In [ ]:
# TODO: Your code here!

Observemos una muestra de las imágenes en el conjunto de datos:

In [ ]:
n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_train[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

### Ejercicio 3: Construyendo un Clasificador de Imágenes

Ahora, construyamos nuestra primera red neuronal. Para ello, creamos una instancia de la clase `keras.models.Sequential`. Esta clase tendrá una lista de capas, cada una de las cuales es una instancia de la clase `keras.layers.Layer`. Cada una de las capas de la red neuronal se pueden añadir con la función `add()`, que recibe una instancia de la clase `keras.layers.Layer`.

Para esta red neuronal vamos a crear primero una capa `Flatten` (disponible en `keras.layers.Flatten(input_shape)`) que convierte las imágenes de 28x28 pixels a una matriz de 784 pixels. Luego, creamos una capa densa (`keras.layers.Dense(units, activation='relu'`) con 300 unidades, otra con 100 unidades, ambas usando la función de activación ReLU y una última capa densa de salida, que debe tener la misma cantidad de unidades que los tipos de clases, en nuestro caso, 10, con función de activación `softmax`. Esta función tiene como salida números en el intervalo de (0, 1), representando una probabilidad.

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

Cada una de las capas además pueden ser pasadas como argumentos de la inicialización del modelo secuencial mediante una lista (`keras.models.Sequential(layers: list)`).

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

Veamos las capas de nuestro modelo usando el atribtuto `layers`:

In [ ]:
model.layers

La función `summary` nos hace un resumen de la estructura de nuestra red neuronal, mostrandonos las dimensiones de salida de cada capa y la cantidad de parámetros a entrenar por cada una de las capas (y la cantidad total).

In [ ]:
model.summary()

Aquellos que tengan `pydot` y `graphviz` instalados, podemos ver una imagen de la red neuronal:	

In [ ]:
keras.utils.plot_model(model, "resources/outputs/my_fashion_mnist_model.png", show_shapes=True)

Cada una de las capas tiene un atributo `name`, veamos el nombre de nuestra 2da capa:

In [ ]:
hidden1 = model.layers[1]
hidden1.name

Mediante el `get_layer` podemos acceder a una capa de nuestro modelo a través de su nombre:

In [ ]:
model.get_layer(hidden1.name) is hidden1

La función `get_weights` de cada una de las capas nos devuelve una matriz de numpy, cada uno de los cuales representa los pesos de la capa y un array que representan los sesgos (*bias*).

In [ ]:
weights, biases = hidden1.get_weights()

Veamos la forma que tienen cada uno de estos parámetros:

In [ ]:
weights

In [ ]:
weights.shape

In [ ]:
biases

In [ ]:
biases.shape

Una vez creada nuestra red neuronal, siempre antes de hacer `fit`, tenemos que definir una función de pérdida y una función de optimización (y opcionalmente una métrica de rendimiento). Esto lo hacemos con la función `compile(loss, optimizer, metrics)`. Como función de pérdida vamos a usar `'sparse_categorical_crossentropy'`, comunmente usada en problemas de clasificación y como función de optimización vamos a usar `'sgd'` (*stochastic gradient descent*). Como medida de rendimiento vamos a usar `'accuracy'`.

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

Esto es equivalente a:

```python
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              metrics=[keras.metrics.sparse_categorical_accuracy])
```

### Ejercicio 4: Entrenando un Clasificador de Imágenes

Ahora, vamos a entrenar a nuestro modelo usando `fit` en nuestro conjunto de datos de entrenamiento, `X_train` y `y_train`. Podemos especificar la cantidad de *epochs* (iteraciones sobre los datos para entrenar el modelo) con el parámetros `epochs` y los datos de validación usados para evaluar el modelo durante su entrenamiento mediante el parámetro `validation_data` (este último espera una tupla, `X_valid` y `y_valid`). 

In [ ]:
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid))

El método `fit` devuelve una instancia del objeto `History`, que guarda información con respecto al proceso de entrenamiento. Por ejemplo:

In [ ]:
history.params

In [ ]:
print(history.epoch)

Los parámetros que se guardan en cada uno de los epochs se pueden ver en `history.history`:

In [ ]:
history.history.keys()

Grafiquemos los distintos resultados obtenidos durante el entrenamiento:

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

### Ejercicio 5: Evaluando un Clasificador de Imágenes

Ahora, realicemos la evaluación de nuestro modelo mediante la función `evaluate`, que recibe como argumentos los datos de prueba y sus etiquetas.

In [ ]:
model.evaluate(X_test, y_test)

Mediante el método `predict`, podemos obtener las predicciones de la red neuronal, que son las probabilidades de pertenenecia a cada una de las clases.

In [ ]:
X_new = X_test[:3]
y_proba = model.predict(X_new)
y_proba.round(2)

Para predecir la clase a la que pertenece, se usa la función `np.argmax`, que nos dará el índice de la clase donde la instancia de entrenamiento tendrá más probabilidad de pertenecer.

In [ ]:
#y_pred = model.predict_classes(X_new) # deprecated
y_pred = np.argmax(model.predict(X_new), axis=-1)
y_pred

Para ver el nombre de la clase a las que pertenece, podemos indexar por el indice de la clase en la lista `class_names` (se recomienda convertirla en un array de numpy para esto):

In [ ]:
np.array(class_names)[y_pred]

Veamos cuales eran las clases reales de estas instancias de prueba (están en `y_test`):

In [ ]:
y_new = y_test[:3]
y_new

Veamos en una imagen cuales eran las clases reales de estas instancias de prueba:

In [ ]:
plt.figure(figsize=(7.2, 2.4))
for index, image in enumerate(X_new):
    plt.subplot(1, 3, index + 1)
    plt.imshow(image, cmap="binary", interpolation="nearest")
    plt.axis('off')
    plt.title(class_names[y_test[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

### Ejercicio 6: Regresión con MLP

Vamos a usar redes neuronales en un problema de regresión, para esto usaremos el dataset de predicción de precios de casas *California housing*. Además, este dataset se divide en conjuntos de entrenamiento, validación y prueba y se normalizan cada una de sus características.

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing(data_home='./resources', download_if_missing=False)

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

Vamos a crear, compilar, ajustar y evaluar una red neuronal en este dataset, como lo vimos anteriormente. La red neuronal en este caso tendrá solamente dos capas, una capa Densa de entrada con 30 neuronas y una capa de salida con una sola neurona.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)

Vamos a analizar el proceso de entrenamiento:

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

Y veamos la predicción:

In [ ]:
y_pred

### Ejercicio 7: API Funcional de Keras

Ahora, vamos a construir una red neuronal más compleja para resolver nuestro problema. No todos los modelos de redes neuronales son simplemente secuenciales. Algunos pueden tener topologías complejas. Algunos pueden tener múltiples entradas y/o múltiples salidas. Para la construcción de redes neuronales complejas, se usa el api funcional de Keras.

Por ejemplo, construyamos esta red neuronal:

In [ ]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.models.Model(inputs=[input_], outputs=[output])

Y veamos su estructura:

In [ ]:
model.summary()

Realicemos el entrenamiento y evaluación de esta red:

In [ ]:
model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
y_pred = model.predict(X_new)

¿Qué sucede si desea enviar diferentes subconjuntos de entidades de entrada a través de rutas anchas o profundas de nuestra red neuronal? Enviaremos 5 características (características 0 a 4) al principio de nuestra red y 6 a través del camino profundo (características 2 a 7). Tenga en cuenta que 3 características pasarán por ambas (características 2, 3 y 4).

In [ ]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

Veamos la topología de nuestra red:

In [ ]:
model.summary()

Y su rendimiento:

In [ ]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

### Ejercicio 8: Salvando y Recuperando Modelos de Keras

Muy a menudo, después del entrenamiento de un modelo, vamos a querer salvarlo para usarlo en el futuro, los modelos de Keras ofrecen varias utilidades en este sentido.

Vamos a crear una red neuronal simple:

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])    

Y a entrenarla.

In [ ]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)

Esta red se puede guardar en un archivo H5 (`.h5`) con la función `save(filepath)` del modelo de Keras.

In [ ]:
model.save("resources/outputs/my_keras_model.h5")

Y se puede cargar con la función `keras.models.load_mode(filepath)`. Esta función devuelve nuestra instancia del objeto `Model`.

In [ ]:
model = keras.models.load_model("resources/outputs/my_keras_model.h5")

El modelo devuelto es totalmente utilizable y se encuentra entrenado, podemos probarlo usando el método `predict`.

In [ ]:
model.predict(X_new)

También podemos guardar solo los pesos del modelo con la función `save_weights(filepath)` del modelo de Keras.

In [ ]:
model.save_weights("resources/outputs/my_keras_weights.h5")

Y cargarlos con la función `load_weights(filepath)` del modelo de Keras.

In [ ]:
model.load_weights("resources/outputs/my_keras_weights.h5")